# kNN:  k Nearest Neighbors

In [49]:
# Imports
import warnings
warnings.simplefilter('ignore')

# %matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn import utils
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler


In [36]:
# Read the csv file into a pandas DataFrame
exoplanet_complete_kNN = pd.read_csv('exoplanet_complete_kNN.csv')
exoplanet_complete_kNN.head()

,koi_disposition,koi_score,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
0,1,1.000,9.488036,170.538750,0.146,2.95750,615.8,2.26,793,93.59,5455,4.467,0.927,291.93423,48.141651,15.347
1,1,0.969,54.418383,162.513840,0.586,4.50700,874.8,2.83,443,9.11,5455,4.467,0.927,291.93423,48.141651,15.347
2,3,0.000,19.899140,175.850252,0.969,1.78220,10829.0,14.60,638,39.30,5853,4.544,0.868,297.00482,48.134129,15.436
3,3,0.000,1.736952,170.307565,1.276,2.40641,8079.2,33.46,1395,891.96,5805,4.564,0.791,285.53461,48.285210,15.597
4,1,1.000,2.525592,171.595550,0.701,1.65450,603.3,2.75,1406,926.16,6031,4.438,1.046,288.75488,48.226200,15.509


## Key for koi_disposition column:
CONFIRMED = 1
CANDIDATE = 2
FALSE POSITIVE = 3


In [37]:
X = exoplanet_complete_kNN[['koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration', 'koi_depth', 'koi_prad', 'koi_teq', 'koi_insol', 'koi_steff', 'koi_slogg', 'koi_steff', 'koi_srad', 'ra', 'dec', 'koi_kepmag']]
y = exoplanet_complete_kNN['koi_disposition'].values.reshape(-1, 1)
target_names = ["1", "2", "3"]
print(X.shape, y.shape)

(7994, 15) (7994, 1)


In [39]:
label_encoder = LabelEncoder()

data = X.copy()

label_encoder.fit(data['koi_period'])
label_encoder.fit(data['koi_time0bk'])
label_encoder.fit(data['koi_impact'])
label_encoder.fit(data['koi_duration'])
label_encoder.fit(data['koi_depth'])
label_encoder.fit(data['koi_prad'])
label_encoder.fit(data['koi_teq'])
label_encoder.fit(data['koi_insol'])
# label_encoder.fit(data['koi_steff'])  #This one produces an error!
label_encoder.fit(data['koi_slogg'])
label_encoder.fit(data['koi_srad'])
label_encoder.fit(data['ra'])
label_encoder.fit(data['dec'])
label_encoder.fit(data['koi_kepmag'])
label_encoder.classes_

array([ 6.966,  6.974,  7.391, ..., 19.319, 19.485, 20.003])

In [45]:
#   All of these produce an error of y something with outcomes, except koi_kepmag - the only that gives an array
#label_encoder.transform(data.koi_period)
#label_encoder.transform(data.koi_time0bk)
#label_encoder.transform(data.koi_impact)
#label_encoder.transform(data.koi_duration)
#label_encoder.transform(data.koi_depth)
#label_encoder.transform(data.koi_prad)
#label_encoder.transform(data.koi_teq)
#label_encoder.transform(data.koi_insol)
# label_encoder.transform(data.koi_steff)
#label_encoder.transform(data.koi_slogg)
#label_encoder.transform(data.koi_srad)
#label_encoder.transform(data.ra)
# label_encoder.transform(data.dec)
label_encoder.transform(data.koi_kepmag)

array([2768, 2768, 2850, ..., 2249, 2803, 2310])

In [55]:
# Reduce data to only the koi_kepmag column
exoplanet_kNN = exoplanet_complete_kNN.drop(['koi_period', 'koi_time0bk','koi_impact', 'koi_duration','koi_depth', 'koi_prad', 'koi_teq', 'koi_insol', 'koi_steff', 'koi_slogg', 'koi_srad', 'ra','dec'], axis=1)
exoplanet_kNN.head()


,koi_disposition,koi_score,koi_kepmag
0,1,1.000,15.347
1,1,0.969,15.347
2,3,0.000,15.436
3,3,0.000,15.597
4,1,1.000,15.509


In [75]:
# Change koi_kepmag column to integers
exoplanet_kNN.koi_kepmag = exoplanet_kNN.koi_kepmag.astype(int)
print(type(exoplanet_kNN.koi_kepmag[2]))

<class 'numpy.int64'>


In [76]:
X = exoplanet_complete_kNN[['koi_kepmag']]
y = exoplanet_complete_kNN['koi_disposition'].values.reshape(-1, 1)
target_names = ["1", "2", "3"]
print(X.shape, y.shape)

(7994, 1) (7994, 1)


In [77]:
# Label Encoder
label_encoder = LabelEncoder()

data = X.copy()
label_encoder.fit(data['koi_kepmag'])

LabelEncoder()

In [78]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)
data_train, data_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [79]:
# MinMaxScalar
#X_minmax = MinMaxScaler().fit(X_train)
data_minmax = MinMaxScaler().fit(data_train)
y_minmax = MinMaxScaler().fit(y_train)

#X_train_minmax = X_minmax.transform(X_train)
data_train_minmax = data_minmax.transform(data_train)
#X_test_minmax = X_minmax.transform(X_test)
data_test_minmax = data_minmax.transform(data_test)
y_train_minmax = y_minmax.transform(y_train)
y_test_minmax = y_minmax.transform(y_test)

In [80]:
print(utils.multiclass.type_of_target(data_train_minmax))

print(utils.multiclass.type_of_target(X_train_minmax.astype('int')))
print(utils.multiclass.type_of_target(y_train_minmax.astype('int')))
print(utils.multiclass.type_of_target(X_test_minmax.astype('int')))
print(utils.multiclass.type_of_target(y_test_minmax.astype('int')))

#print(utils.multiclass.type_of_target(encoded))

continuous
binary
binary
binary
binary


In [82]:
# Loop through different k values to see which has the highest accuracy
# Note: We only use odd numbers because we don't want any ties
train_scores = []
test_scores = []
for k in range(1, 20, 2):
    knn = KNeighborsClassifier(n_neighbors=k)
    #knn.fit(X_train_minmax, y_train_minmax)
    #train_score = knn.score(X_train_minmax, y_train_minmax)
    #test_score = knn.score(X_test_minmax, y_test_minmax)
    knn.fit(data_train_minmax, y_train_minmax)
    train_score = knn.score(data_train_minmax, y_train_minmax)
    test_score = knn.score(data_test_minmax, y_test_minmax)
    train_scores.append(train_score)
    test_scores.append(test_score)
    print(f"k: {k}, Train/Test Score: {train_score:.3f}/{test_score:.3f}")

ValueError: Unknown label type: 'continuous'

In [ ]:
knn = KNeighborsClassifier(n_neighbors=9)
knn.fit(X_train_scaled, y_train)
print('k=9 Test Acc: %.3f' % knn.score(X_test_scaled, y_test))